In [1]:
# from typing import Callable, Optional
# import torch
# from torch import nn
# import torch.nn.functional as F
# from torch_geometric.data import DataLoader, Dataset
# from torch_geometric.nn import VGAE, GATConv, GCNConv, GAE
# from torch_geometric.utils import train_test_split_edges
# from typing import Any, List
# import os
# import os.path as osp
# from copy import copy
# from tqdm import tqdm
# import warnings
# warnings.filterwarnings("ignore")
# import pickle

# #empty cuda cache
# torch.cuda.empty_cache()



# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# class graph_dataset(Dataset):
#     def __init__(self, 
#                  files_list: List[str],
#                  root: str | None = None, 
#                  transform: Callable[..., Any] | None = None, 
#                  pre_transform: Callable[..., Any] | None = None, 
#                  pre_filter: Callable[..., Any] | None = None, 
#                  log: bool = True):
#         super().__init__(root, transform, pre_transform, pre_filter, log)
#         self.files_list = files_list
        
        
#     @property
#     def raw_file_names(self):
#         return self.files_list
    
#     @property
#     def processed_file_names(self):
#         return self.files_list
    
#     def len(self):
#         return len(self.processed_file_names)

#     def get(self, idx):
#         data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
#         return data
    
# class VariationalGCNEncoder(torch.nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(VariationalGCNEncoder, self).__init__()
#         self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
#         self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
#         self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index).relu()
#         return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
    
    
# class VariationalGATEncoder(torch.nn.Module):
#     def __init__(self, in_channels, out_channels, hidden_channels):
#         super(VariationalGATEncoder, self).__init__()
#         self.conv1 = GATConv(in_channels, hidden_channels, heads=1, dropout=0.2) # cached only for transductive learning
#         self.conv_mu = GATConv(hidden_channels, out_channels, heads=1, dropout=0.2)
#         self.conv_logstd = GATConv(hidden_channels, out_channels, heads=1, dropout=0.2)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index).relu()
#         return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
    
    
# def train(model, optimizer, g_data):
#     model.train()
#     optimizer.zero_grad()
#     z = model.encode(g_data.x, g_data.train_pos_edge_index)
#     loss = model.recon_loss(z, g_data.train_pos_edge_index)
    
#     loss = loss + (1 / g_data.num_nodes) * model.kl_loss()  # new line
#     loss.backward()
#     optimizer.step()
        
#     return float(loss) 


# def test(model, g_data):
#     # print(g_data.x)
#     model.eval()
#     with torch.no_grad():
#         z = model.encode(g_data.x, g_data.train_pos_edge_index)
#     return model.test(z, g_data.test_pos_edge_index, g_data.test_neg_edge_index)


# def train_vgae(model, datas, optimizer):
#     # print(datas)
#     total_loss = 0
#     total_ap, total_auc = 0, 0
#     for data in tqdm(datas, leave=False):

#         #if graph is None, skip
#         if data is None:
#             continue

#         if(data.edge_index is None):
#             continue

#         if(data.x is None):
#             continue

#         if(data.edge_index.shape[0] == 0 or data.x.shape[1] == 0):
#             continue
        
#         # path = f"./graphs/{data}"
#         g_data = data.to(DEVICE)
#         # print(g_data.edge_index.shape)
        
#         # g_data=torch.load(open(f"graphs/{data}", "rb")).to(DEVICE)

#         # print(g_data.edge_index.shape)
#         g_data.edge_index = torch.vstack([g_data.edge_index[1, :], g_data.edge_index[0, :]])
#         g_data = train_test_split_edges(g_data)


        



#         # print(g_data)
#         loss = train(model, optimizer, g_data)

#         if g_data.test_pos_edge_index.size(1) == 0:
#             # print(f"No edges to train/test on for graph {g_data}")
#             continue  # Skip this graph

#         # print("here")
#         # print(loss)
#         total_loss += loss
#         auc, ap = test(model, g_data)
#         total_auc += auc
#         total_ap += ap
        
#     return total_loss / len(datas), total_ap / len(datas), total_auc / len(datas)



In [85]:
# TODO: Need to load our connectome dataset
datas=graphs
# Define the graph encoder model
class GraphEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, encoding_dim):
        super(GraphEncoder, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, encoding_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x=F.dropout(x,p=0.2)
        z = self.conv2(x, edge_index)
        z=F.dropout(z,p=0.2)
        return z

# Define the graph decoder model
class GraphDecoder(torch.nn.Module):
    def __init__(self, encoding_dim, hidden_dim, output_dim):
        super(GraphDecoder, self).__init__()
        self.conv1 = GCNConv(encoding_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, z, edge_index):
        x = F.relu(self.conv1(z, edge_index))
        x=F.dropout(x,p=0.2)
        x = self.conv2(x, edge_index)
        x=F.dropout(x,p=0.2)

        return x

    
# Create instances of the graph encoder and decoder models
input_dim = 20
hidden_dim = 16
encoding_dim = 16
output_dim = input_dim
encoder = GraphEncoder(input_dim, hidden_dim, encoding_dim)
decoder = GraphDecoder(encoding_dim, hidden_dim, output_dim)

In [90]:
import glob

files=glob.glob("./Seperated_dataset/*")
all_graphs=[]
for dataset_folder in files:
    # print(dataset_folder)
    # print(dataset_folder)
    graph_path=f"{dataset_folder}/train_feature.pt"
    graphs=torch.load(graph_path)
    print(len(graphs))
    all_graphs+=graphs
    
    

1900
760
606
720
1874
892
444
2196
676
856
2284
1074


In [91]:
graphs_all=all_graphs

In [97]:
graphs_all

[Data(x=[13, 20], edge_index=[2, 26], edge_attr=[26, 19]),
 Data(x=[26, 20], edge_index=[2, 60], edge_attr=[60, 19]),
 Data(x=[19, 20], edge_index=[2, 42], edge_attr=[42, 19]),
 Data(x=[13, 20], edge_index=[2, 24], edge_attr=[24, 19]),
 Data(x=[9, 20], edge_index=[2, 16], edge_attr=[16, 19]),
 Data(x=[9, 20], edge_index=[2, 18], edge_attr=[18, 19]),
 Data(x=[9, 20], edge_index=[2, 16], edge_attr=[16, 19]),
 Data(x=[6, 20], edge_index=[2, 10], edge_attr=[10, 19]),
 Data(x=[78, 20], edge_index=[2, 162], edge_attr=[162, 19]),
 Data(x=[16, 20], edge_index=[2, 30], edge_attr=[30, 19]),
 Data(x=[13, 20], edge_index=[2, 24], edge_attr=[24, 19]),
 Data(x=[11, 20], edge_index=[2, 22], edge_attr=[22, 19]),
 Data(x=[24, 20], edge_index=[2, 46], edge_attr=[46, 19]),
 Data(x=[13, 20], edge_index=[2, 28], edge_attr=[28, 19]),
 Data(x=[10, 20], edge_index=[2, 16], edge_attr=[16, 19]),
 Data(x=[10, 20], edge_index=[2, 18], edge_attr=[18, 19]),
 Data(x=[14, 20], edge_index=[2, 26], edge_attr=[26, 19]),

In [103]:
datas=graphs_all
# Define the loss function
# loss_fn = torch.nn.BCEWithLogitsLoss()
loss_fn = torch.nn.MSELoss()  # If the features are continuous


# TODO: Loss function may need to be changed. Not sure if BCE is the right one

# Define the optimizer
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.01)

# Training loop
def train():

    encoder.train()
    decoder.train()
    optimizer.zero_grad()
    total_loss=0
    
    for graph in tqdm(datas):
        # print(graph)
        if graph is None:
            continue
        if graph.edge_index.shape[0] == 0 :
            continue
        x = graph.x
        edge_index = graph.edge_index
        z = encoder(x, edge_index)
        reconstructed_x = decoder(z, edge_index)
        loss = loss_fn(reconstructed_x, x)
        total_loss += loss.item()
        loss.backward()
    optimizer.step()
    return total_loss / len(datas)

# Evaluation loop
def test():
    encoder.eval()
    decoder.eval()
    total_loss=0
    with torch.no_grad():
        for graph in tqdm(datas):
            if graph is None:
                continue
            if graph.edge_index.shape[0] == 0 :
                continue
            x = graph.x
            edge_index = graph.edge_index
            z = encoder(x, edge_index)
            reconstructed_x = decoder(z, edge_index)
            loss = loss_fn(reconstructed_x, x)
            total_loss += loss.item()
    return total_loss / len(datas)

# Train and evaluate the model
for epoch in range(200):

    loss = train()
    if(epoch%50==0):
        torch.save(encoder.state_dict(), f'encoder_weights_epoch_{epoch+1}.pth')
        torch.save(decoder.state_dict(), f'decoder_weights_epoch_{epoch+1}.pth')


    #if epoch % 10 == 0:
    eval_loss= test()
    print(f'Epoch: {epoch}, Loss: {loss:.4f}, Eval Loss: {eval_loss:.4f}')

100%|██████████| 14282/14282 [00:12<00:00, 1187.38it/s]


Epoch: 0, Loss: 8.1948, Eval Loss: 7.1810


100%|██████████| 14282/14282 [00:12<00:00, 1183.29it/s]


Epoch: 1, Loss: 7.1852, Eval Loss: 6.4695


100%|██████████| 14282/14282 [00:12<00:00, 1140.06it/s]


...

Epoch: 165, Loss: 3.4454, Eval Loss: 3.4496


100%|██████████| 14282/14282 [00:12<00:00, 1123.42it/s]


Epoch: 166, Loss: 3.4552, Eval Loss: 3.4790


 76%|███████▌  | 10869/14282 [00:13<00:04, 809.02it/s]


KeyboardInterrupt: 

In [104]:


# graph_encoder = VariationalGATEncoder(20, 8,8)
# # graph_encoder = VariationalGATEncoder(128, 16, 32)
# model = VGAE(graph_encoder).to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# epochs = 10
# best_auc = 0
# for i in (range(epochs)):
#     print(i)
#     # print(nr_er_graphs)
#     loss, mean_ap, mean_auc = train_vgae(model, graphs_all, optimizer)
#     print(f"Epoch {i}, Loss: {loss}, Mean AP: {mean_ap}, Mean AUC: {mean_auc}")
#     if mean_auc > best_auc:
#         torch.save(
#             model.state_dict(),
#             "vgae_models/best_autoencoder.pth"
#         )
#         best_auc = mean_auc


In [105]:
# g_data=graphs_all[0]

# model.eval()
# with torch.no_grad():
#     z = model.encode(g_data.x, g_data.train_pos_edge_index)

# performance testing

In [106]:
nr_ahr_data=torch.load("Seperated_dataset/nr-ahr/train_feature.pt")
nr_ahr_label=torch.load("Seperated_dataset/nr-ahr/train_labels.pt")


nr_ahr_test_data=torch.load("Seperated_dataset/nr-ahr/test_feature.pt")
nr_ahr_test_label=torch.load("Seperated_dataset/nr-ahr/test_labels.pt")

In [109]:
encoder.to(DEVICE)

GraphEncoder(
  (conv1): GCNConv(20, 16)
  (conv2): GCNConv(16, 16)
)

In [134]:
nr_ahr_data

[Data(x=[13, 20], edge_index=[2, 26], edge_attr=[26, 19]),
 Data(x=[26, 20], edge_index=[2, 60], edge_attr=[60, 19]),
 Data(x=[19, 20], edge_index=[2, 42], edge_attr=[42, 19]),
 Data(x=[13, 20], edge_index=[2, 24], edge_attr=[24, 19]),
 Data(x=[9, 20], edge_index=[2, 16], edge_attr=[16, 19]),
 Data(x=[9, 20], edge_index=[2, 18], edge_attr=[18, 19]),
 Data(x=[9, 20], edge_index=[2, 16], edge_attr=[16, 19]),
 Data(x=[6, 20], edge_index=[2, 10], edge_attr=[10, 19]),
 Data(x=[78, 20], edge_index=[2, 162], edge_attr=[162, 19]),
 Data(x=[16, 20], edge_index=[2, 30], edge_attr=[30, 19]),
 Data(x=[13, 20], edge_index=[2, 24], edge_attr=[24, 19]),
 Data(x=[11, 20], edge_index=[2, 22], edge_attr=[22, 19]),
 Data(x=[24, 20], edge_index=[2, 46], edge_attr=[46, 19]),
 Data(x=[13, 20], edge_index=[2, 28], edge_attr=[28, 19]),
 Data(x=[10, 20], edge_index=[2, 16], edge_attr=[16, 19]),
 Data(x=[10, 20], edge_index=[2, 18], edge_attr=[18, 19]),
 Data(x=[14, 20], edge_index=[2, 26], edge_attr=[26, 19]),

In [126]:
encoded_datas=[]
actual_labels=[]

encoded_datas_test=[]
actual_labels_test=[]


for data,label in tqdm(zip(nr_ahr_data,nr_ahr_label)):
    encoder.eval()
    with torch.no_grad():
        # print(data)
        if(data.edge_index is None):
            continue
        if(data.edge_index.shape[0] == 0 or data.x.shape[1] == 0):
            continue    
        g_data=data.to(DEVICE)

        z = encoder(g_data.x, g_data.edge_index)
        encoded_datas.append(z)
        actual_labels.append(label)


for data,label in tqdm(zip(nr_ahr_test_data,nr_ahr_test_label)):
    encoder.eval()
    with torch.no_grad():
        # print(data)
        if(data.edge_index is None):
            continue
        if(data.edge_index.shape[0] == 0 or data.x.shape[1] == 0):
            continue    
        g_data=data.to(DEVICE)


        z = encoder(g_data.x, g_data.edge_index)
        encoded_datas_test.append(z)
        actual_labels_test.append(label)

1900it [00:02, 920.30it/s]
608it [00:00, 954.12it/s] 


In [127]:
encoded_datas[0].shape

torch.Size([13, 16])

In [128]:
for i in range(len(encoded_datas)):
    encoded_datas[i]=torch.mean(encoded_datas[i],dim=0)
    # detach at cpu
    encoded_datas[i]=encoded_datas[i].detach().cpu().numpy()
    encoded_datas[i]=torch.tensor(encoded_datas[i])


for i in range(len(encoded_datas_test)):
    encoded_datas_test[i]=torch.mean(encoded_datas_test[i],dim=0)
    # detach at cpu
    encoded_datas_test[i]=encoded_datas_test[i].detach().cpu().numpy()
    encoded_datas_test[i]=torch.tensor(encoded_datas_test[i])
    

In [130]:
for i in range(len(encoded_datas)):
    # extend the dimension to 1, to make it 2D
    encoded_datas[i]=torch.unsqueeze(encoded_datas[i],0)
    

for i in range(len(encoded_datas_test)):
    # extend the dimension to 1, to make it 2D
    encoded_datas_test[i]=torch.unsqueeze(encoded_datas_test[i],0)
    

In [1]:
encoded_datas[0].shape

NameError: name 'encoded_datas' is not defined

In [133]:
import numpy as np
#create a dataloader for the encoded data
class encoded_dataset(Dataset):
    def __init__(self, 
                 data: List[torch.Tensor],
                 labels: List[int],
                 root: str | None = None, 
                 transform: Callable[..., Any] | None = None, 
                 pre_transform: Callable[..., Any] | None = None, 
                 pre_filter: Callable[..., Any] | None = None, 
                 log: bool = True):
        super().__init__(root, transform, pre_transform, pre_filter, log)
        self.data = data
        self.labels = labels
        
    def len(self):
        return len(self.data)

    def get(self, idx):
        # Ensure that data and labels are torch.Tensors
        data_tensor = torch.Tensor(self.data[idx]) if isinstance(self.data[idx], np.ndarray) else self.data[idx]
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.long) 
        # print(data_tensor)
        # print(label_tensor)
        return data_tensor, label_tensor

class EncodedClassifier(nn.Module):
    def __init__(self, in_features, out_features):
        super(EncodedClassifier, self).__init__()
        
        # Adjusting for input shape (batch_size, 1, 16)
        self.conv1 = nn.Conv1d(1, 128, kernel_size=3, padding=1)  # input channel is 1
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(256)
        self.pool = nn.MaxPool1d(2)  # Downsample, resulting in halving the sequence length
        
        # After two pooling operations on an input of length 16:
        # First pooling -> 16 / 2 = 8
        # Second pooling -> 8 / 2 = 4
        # Therefore, the flattened size before the fully connected layer is 256 * 4
        self.fc1 = nn.Linear(256 * 4, 512)  # Adjusted the size for the new flattened output
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, out_features)
        
    def forward(self, x):
        # x shape is (batch_size, 1, 16)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)  # x shape becomes (batch_size, 128, 8)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)  # x shape becomes (batch_size, 256, 4)
        
        # Flatten before passing to the dense layer
        x = x.view(x.size(0), -1)  # Flatten to (batch_size, 256*4)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # No activation, assuming we're using BCEWithLogitsLoss or similar
        
        return x



#split the data into train and valid
    
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(encoded_datas, actual_labels, test_size=0.1, random_state=42)

train_dataset=encoded_dataset(X_train,y_train)
test_dataset=encoded_dataset(X_test,y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)



test_final_dataset=encoded_dataset(encoded_datas_test,actual_labels_test)
test_final_loader = DataLoader(test_final_dataset, batch_size=32, shuffle = False)

#train the classifier

classifier = EncodedClassifier(16, 1).to(DEVICE)

optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

epochs = 200

#calculate the accuracy and roc_auc_score for each epoch. Save the best model based on the roc_auc_score

from sklearn.metrics import accuracy_score, roc_auc_score

best_auc = 0

for epoch in range(epochs):
    classifier.train()
    train_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = classifier(data)
        loss = criterion(output, target.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
    train_loss = train_loss / len(train_loader.dataset)
    # print(f"Epoch {epoch} - Train Loss: {train_loss}")
    
    classifier.eval()
    test_loss = 0.0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in test_final_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = classifier(data)
            loss = criterion(output, target.unsqueeze(1).float())
            test_loss += loss.item() * data.size(0)
            y_true += target.cpu().numpy().tolist()
            y_pred += torch.sigmoid(output).cpu().numpy().tolist()
    test_loss = test_loss / len(test_final_loader.dataset)
    y_pred = [i[0] for i in y_pred]
    auc = roc_auc_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, [1 if i > 0.5 else 0 for i in y_pred])
    # print(f"Epoch {epoch} - Test Loss: {test_loss} - AUC: {auc} - Accuracy: {accuracy}")
    if auc > best_auc:
        print(f"Saving best model with AUC: {auc}")
        torch.save(
            classifier.state_dict(),
            "./Seperated_dataset/nr-ahr/best_classifier_gae.pt"
        )
        best_auc = auc

#load the best classifier

Saving best model with AUC: 0.842814803157867
Saving best model with AUC: 0.8471162168541979


In [142]:
#load the best classifier
classifier = EncodedClassifier(16, 1).to(DEVICE)
classifier.load_state_dict(torch.load("./Seperated_dataset/nr-ahr/best_classifier_gae.pt"))




classifier.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for data, target in test_final_loader:
        # print("hi")
        data, target = data.to(DEVICE), target.to(DEVICE)
        output = classifier(data)
        # print(output)
        y_true += target.cpu().numpy().tolist()
        y_pred += torch.sigmoid(output).cpu().numpy().tolist()
        # print(y_pred)

y_pred = [i[0] for i in y_pred]
#save the y_pred to a npy file in the corresponding folder
np.save("./Seperated_dataset/nr-ahr/pred_gae.npy",y_pred)
auc = roc_auc_score(y_true, y_pred)
accuracy = accuracy_score(y_true, [1 if i > 0.5 else 0 for i in y_pred])

print(f"AUC: {auc} - Accuracy: {accuracy}")


AUC: 0.8471162168541979 - Accuracy: 0.6990131578947368
